In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit 
from sklearn.metrics import mean_absolute_error



In [ ]:
def select_drug(data, drug, gene):
    drug = drug.split()[0]
    matching_columns = []


    filtered_data = data[data['Probe_id'] == gene]
    
    
    # Find matching columns for the drug
    for idx,col in enumerate(data.columns):
        if drug in col.split():
            matching_columns.append(filtered_data[col])
    
   
    if matching_columns:
        result = pd.concat(matching_columns, axis=1)
        return result.values.flatten().astype(float)
    else:
        return np.array([])

In [ ]:
def dose_level(data, drug):
    '''catch dose and map to numerical value'''
    drug=drug.split()[0]
    doses = []
    dose_mapping = {'L': 1, 'M': 3, 'H': 10} 
    for col in data.columns:
        if col.split()[0] == drug:
            dose = col.split()[1]
            if dose in dose_mapping:
                doses.append(dose_mapping[dose])
                
    return doses
                

In [ ]:

def logistic_function(x, b, e, c, d):
        return c + (d - c) / (1 + np.exp(b * (np.log(x) - np.log(e))))

In [ ]:
Genes = []
R_squared = []
Total_result = []



for gene in data['Probe_id']:
    processed_drugs = set()
    for idx, drug in enumerate(Filtered_toxicology.columns[2:]):
        base_drug = drug.split()[0]
        
        if base_drug in processed_drugs:
            continue
        
        response = np.array(select_drug(Filtered_toxicology, drug, gene)).astype(float)
        dose = np.array(dose_level(Filtered_toxicology, drug)).astype(float)
        
        # Check for NaN values or insufficient data points
        if np.isnan(response).any() or np.isnan(dose).any() or len(response) < 3:
            continue
        
        # To have better estimation
        c_fixed = min(response)
        
        # Initial guesses
        b_initial = 1
        e_initial = np.median(dose)
        d_initial = np.max(response)
        
        initial_guesses = [b_initial, e_initial, d_initial]
        
        # Skip if we encounter NaN in our initial guesses
        if np.isnan(initial_guesses).any():
            continue
        
        try:
            params, params_covariance = curve_fit(
                lambda x, b, e, d: logistic_function(x, b, e, c_fixed, d),
                dose, response, p0=initial_guesses, method='dogbox', maxfev=10000000
            )
            b_fitted, e_fitted, d_fitted = params
            predicted_values = logistic_function(dose, b_fitted, e_fitted, c_fixed, d_fitted)

            residuals = response - predicted_values

            ss_tot = np.sum((response - np.mean(response))**2)
            ss_res = np.sum(residuals**2)
            r_squared = 1 - (ss_res / ss_tot)

            # Generate fitted values
            x_fitted = np.linspace(min(dose), max(dose), 100)
            y_fitted = logistic_function(x_fitted, b_fitted, e_fitted, c_fixed, d_fitted)

            first_derivative = np.gradient(y_fitted, x_fitted)
            second_derivative = np.gradient(first_derivative, x_fitted)

            # Store R_squared if criteria met
            if r_squared > 0.99 and np.max(first_derivative) > 0.1:
                R_squared.append(r_squared)
                if gene not in Genes and len(R_squared)> 1:
                    Genes.append(gene)
          
            
       
            
            plt.scatter(dose, response)
            plt.plot(x_fitted, y_fitted, label=f'Dose-Response Curve for {drug}')   
            plt.xlabel('Dose')
            plt.ylabel('Response')
            plt.legend()
            plt.show()
            
            
            
        
        except RuntimeError as e:
            print(f"{drug} could not be fitted. Error: {e}")
            

outpath = 'Gene_Selection_with0.1.csv'
result = pd.DataFrame(np.array(Genes), columns=["Gene"])
result.to_csv(outpath, index=False)

In [ ]:
def dose_response_curve(data):
    Genes = []
    R_squared = []

    for gene in data['Probe_id']:
        processed_drugs = set()
        
        for idx, drug in enumerate(data.columns[2:]):
            base_drug = drug.split()[0]
            
            if base_drug in processed_drugs:
                continue
            
            response = np.array(select_drug(data, drug, gene)).astype(float)
            dose = np.array(dose_level(data, drug)).astype(float)
            
            # Check for NaN values or insufficient data points
            if np.isnan(response).any() or np.isnan(dose).any() or len(response) < 3:
                continue
            
            # Initial parameter guesses
            c_fixed = np.min(response)
            b_initial = 1
            e_initial = np.median(dose)
            d_initial = np.max(response)
            initial_guesses = [b_initial, e_initial, d_initial]
            
            if np.isnan(initial_guesses).any():
                continue
            
            try:
                params, _ = curve_fit(
                    lambda x, b, e, d: logistic_function(x, b, e, c_fixed, d),
                    dose, response, p0=initial_guesses, method='dogbox', maxfev=10000000
                )
                b_fitted, e_fitted, d_fitted = params
                predicted_values = logistic_function(dose, b_fitted, e_fitted, c_fixed, d_fitted)
                
                residuals = response - predicted_values
                ss_tot = np.sum((response - np.mean(response))**2)
                ss_res = np.sum(residuals**2)
                r_squared = 1 - (ss_res / ss_tot)
                
                # Generate fitted values
                x_fitted = np.linspace(np.min(dose), np.max(dose), 100)
                y_fitted = logistic_function(x_fitted, b_fitted, e_fitted, c_fixed, d_fitted)
                
                first_derivative = np.gradient(y_fitted, x_fitted)
                second_derivative = np.gradient(first_derivative, x_fitted)
                
                # Store R_squared if criteria met
                if r_squared > 0.99 and np.max(first_derivative) > 0.1:
                    R_squared.append(r_squared)
                    if gene not in Genes and len(R_squared) > 1:
                        Genes.append(gene)
                        
                # Plotting
                plt.scatter(dose, response, label=f'{drug} Data')
                plt.plot(x_fitted, y_fitted, label=f'Dose-Response Curve for {drug}')
                plt.xlabel('Dose')
                plt.ylabel('Response')
                plt.title(f'Dose-Response Curve for {drug}')
                plt.legend()
                plt.show()
                
            except RuntimeError as e:
                print(f"{drug} could not be fitted. Error: {e}")
    
    outpath = 'Gene_Selection_with0.1.csv'
    result = pd.DataFrame(Genes, columns=["Gene"])
    result.to_csv(outpath, index=False)
